<a href="https://colab.research.google.com/github/Clarison/CashFlowAnalysis/blob/main/ConvertPDFtoEXcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tabula-py pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.3 MB/s eta 0:00:00


In [2]:
import tabula
import pandas as pd
import numpy as np

In [26]:
pdf_file = "/content/apple.pdf"

In [76]:
# Extract tables from the PDF
tables = tabula.read_pdf(pdf_file, pages="all")

# Combine tables into a single DataFrame (if multiple tables)
df1 = pd.concat(tables)

# Use the first row as column names
df1.columns = df1.iloc[0]

# Drop the first row (header row)
df1 = df1.drop(0)

# Reset the index
df1.reset_index(drop=True, inplace=True)

df1.columns = ['Category', '2022', '2021', '2020']

df1['2022'] = df1['2022'].str.replace('$', '').str.replace(',', '').str.replace(r'\((.*?)\)', r'-\1').astype(float)
df1['2021'] = df1['2021'].str.replace('$', '').str.replace(',', '').str.replace(r'\((.*?)\)', r'-\1').astype(float)
df1['2020'] = df1['2020'].str.replace('$', '').str.replace(',', '').str.replace(r'\((.*?)\)', r'-\1').astype(float)

# Initialize a new column 'Level' to track hierarchy
df1['Level'] = np.nan

# Set the hierarchy level based on NaN values in the 'Category' column
current_level = 0
for i, row in df1.iterrows():
    if pd.isna(row['2021']):
        current_level += 1
    df1.at[i, 'Level'] = current_level



# Find the unique categories where '2022' is null for each level
null_2022_categories = df1[df1['2022'].isnull()].groupby('Level')['Category'].first().reset_index()

# Merge the unique categories back to the original DataFrame
df1 = df1.merge(null_2022_categories, on='Level', suffixes=('', '_Parent'), how='left')

# Rename the 'Category_Parent' column to 'Parent Category' and fill NaN values with 'Category'
df1['Parent Category'] = df1['Category_Parent'].fillna(df1['Category'])

# Drop the 'Category_Parent' column
df1.drop('Category_Parent', axis=1, inplace=True)

df1['Parent Category'] = df1['Parent Category'].str.replace(':', '')
# Reorder columns to have 'Parent Category' as the first column
df1 = df1[['Parent Category'] + [col for col in df1 if col != 'Parent Category']]

df1 = df1.iloc[:, :-1]


df1 = df1.dropna(subset=['2022'])

<ipython-input-76-f1de1c1f7664>:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['2022'] = df1['2022'].str.replace('$', '').str.replace(',', '').str.replace(r'\((.*?)\)', r'-\1').astype(float)
<ipython-input-76-f1de1c1f7664>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['2022'] = df1['2022'].str.replace('$', '').str.replace(',', '').str.replace(r'\((.*?)\)', r'-\1').astype(float)
<ipython-input-76-f1de1c1f7664>:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['2021'] = df1['2021'].str.replace('$', '').str.replace(',', '').str.replace(r'\((.*?)\)', r'-\1').astype(float)
<ipython-input-76-f1de1c1f7664>:19: Futu

In [77]:
df1

,Parent Category,Category,2022,2021,2020
1,Net sales,Products,316199.00,297392.00,220747.00
2,Net sales,Services,78129.00,68425.00,53768.00
3,Net sales,Total net sales,394328.00,365817.00,274515.00
5,Cost of sales,Products,201471.00,192266.00,151286.00
6,Cost of sales,Services,22075.00,20715.00,18273.00
7,Cost of sales,Total cost of sales,223546.00,212981.00,169559.00
8,Cost of sales,Gross margin,170782.00,152836.00,104956.00
10,Operating expenses,Research and development,26251.00,21914.00,18752.00
11,Operating expenses,"Selling, general and administrative",25094.00,21973.00,19916.00
12,Operating expenses,Total operating expenses,51345.00,43887.00,38668.00
